# Carga de datos
Utilizamos un conjunto de datos de alrededor de 37000 reseñas de videos de Amazon Instant Video y 1700000 reseñas de películas y programas de televisión, todas obtenidas del sitio web: http://jmcauley.ucsd.edu/data/amazon/. Tenga en cuenta que hay conjuntos de datos mucho más grandes disponibles en el mismo sitio. Podemos esperar resultados mejores y más consistentes en conjuntos de datos más grandes (como reseñas de libros). Tenga en cuenta que estos conjuntos de datos están comprimidos (gzip) y están en formato JSON, cada línea representa una revisión y cada línea es su propio objeto JSON.

## [NLTK](https://www.nltk.org/)

In [ ]:
# ! pip install --user -U nltk

In [ ]:
%%time
import gzip
import json
import string

import nltk # imports the natural language toolkit -- una libreria que nos sirve para hacer procesos de NLP
import pandas as pd
import plotly


# https://www.nltk.org/api/nltk.tokenize.punkt.html
nltk.download('punkt')

In [ ]:
data_uri_raw: str = "https://github.com/Andru-1987/csv_files_ds/raw/refs/heads/main/reviews_Amazon_Instant_Video_5.json.gz"
data_uri_movies: str = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_5.json.gz"

In [ ]:
instant_video = pd.read_json(data_uri_raw, lines=True, compression='gzip')

In [ ]:
len(instant_video)

In [ ]:
instant_video.head(10)

In [ ]:
%%time
# podemos decirle a pandas que nuestro archivo esta en formato gzip y realizara el proceso de descompresion 
# para esto usamos el atributo `lines=True` para indicar que cada linea en el archivo tiene su propio JSON object
#instant_video = pd.read_json("reviews_Amazon_Instant_Video_5.json.gz", lines=True, compression='gzip')
# ----------
# El archivo Películas y TV es muy grande. Si tienes problemas para cargarlo, puedes cargar solo los primeros
# 100,000 reseñas usando 'chunksize' (elimine el comentario de la línea con 'chunksize' y comente la línea
# después de lo que carga el archivo completo en `movies_tv`). Todo el análisis puede ser
# hecho de la misma manera utilizando solo el subconjunto de revisiones, pero algunos de los resultados pueden ser diferentes de los ejemplos.
# ----------
movies_tv = next(pd.read_json(data_uri_movies, lines=True, compression='gzip', chunksize=10_000))
movies_tv.sample(10)

Miremos algunos de los atributos

* **reviewerID:** A unique ID to identify the author of the review.
* **asin:** The ["Amazon Standard Identification Number"](https://www.amazon.com/gp/seller/asin-upc-isbn-info.html) which provides more information about the exact product and version.
* **reviewerName:** The username chosen by the reviewer.
* **helpful:** A record of how many users indicated that the review was helpful/not helpful.
* **reviewText:** The full text of the review.
* **overall:** The overall rating (1-5) left by the reviewer.
* **summary:** A short version of the review, used as the title.
* **unixReviewTime:** The date that the review was created, in [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time) format.
* **reviewTime:** A human readable date giving the day, month, and year.

## INFORMACION DE LA DATA

In [ ]:
movies_tv['datetime'] = pd.to_datetime(movies_tv['reviewTime'], format="%m %d, %Y")
instant_video['datetime'] = pd.to_datetime(instant_video['reviewTime'], format="%m %d, %Y")

## CLEAN DATAFRAME

In [ ]:
movies_tv = movies_tv.drop(columns = ['reviewerID', 'asin', 'reviewerName', 'reviewTime'])
instant_video = instant_video.drop(columns = ['reviewerID', 'asin', 'reviewerName', 'reviewTime'])

In [ ]:
movies_tv.datetime.value_counts().head(20)

In [ ]:
movies_tv.datetime.dt.year.hist()

In [ ]:
instant_video.hist()
movies_tv.hist()

Notamos eso:

1. La mayoría de las reseñas son buenas; de hecho, ¡más de la mitad son perfectas (5/5 estrellas) en ambos conjuntos de datos!
2. Casi todas las revisiones de videos instantáneos son extremadamente recientes. Será muy difícil detectar tendencias sin corregir esto. Las reseñas de TV Movie están sustancialmente más dispersas (aunque también incluyen un pico reciente).

In [ ]:
instant_video['year'] = instant_video.datetime.dt.year
instant_video.groupby(['year']).agg({'overall': 'mean', 'year': 'count'})

In [ ]:
movies_tv['year'] = movies_tv.datetime.dt.year
movies_tv.groupby(['year']).agg({'overall': 'mean', 'year': 'count'})

# Promedio de ratings en el tiempo

In [ ]:
result = instant_video.groupby(instant_video.datetime.dt.year).agg({'overall': 'mean', 'overall': 'count'})
result

In [ ]:
instant_video.groupby(instant_video.datetime.dt.year)['overall'].mean().to_frame().plot(kind="bar")
movies_tv.groupby(movies_tv.datetime.dt.year)['overall'].mean().to_frame().plot(kind="bar")

Notamos eso:

1. No hay tendencias notables de cambio de calificaciones con el tiempo.

2. Hay algunos años atípicos con calificaciones más altas, pero estos son años en los que la cantidad de revisiones es muy pequeña, lo que hace que el promedio sea menos confiable.

In [ ]:
instant_video['length'] = instant_video['reviewText'].apply(len)
instant_video.groupby(instant_video.datetime.dt.year)['length'].mean().to_frame().plot(kind="bar")

movies_tv['length'] = movies_tv['reviewText'].apply(len)
movies_tv.groupby(movies_tv.datetime.dt.year)['length'].mean().to_frame().plot(kind="bar")

Observamos una aparente tendencia al alza en la duración de las reseñas durante la década de 1990 y principios de la de 2000, seguida de una tendencia a la baja. Debido a que ambos conjuntos de datos tienen revisiones más recientes, la tendencia a la baja en la duración de la revisión es más confiable.

# FRECUENCIA DE PALABRAS
Encontremos las diez palabras sin stopwrods que aparecen con mayor frecuencia en: (i) todas las reseñas, (ii) reseñas positivas, (iii) reseñas negativas. ¿Te sorprenden los resultados? ¿Por qué o por qué no?

In [ ]:
import nltk
import re
from collections import Counter
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def split_reviews(df, threshold):
    df_lower = df[df['overall'] < threshold].copy()
    df_upper = df[df['overall'] >= threshold].copy()

    return df_lower, df_upper

def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    words = text.split()  # Tokenize
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return words  # Return a list of words


def get_top_n_words(df, col, n):
    all_words = [word for review in df[col] for word in review.split()]  # Ensure words are split correctly
    word_freq = Counter(all_words)
    table = pd.DataFrame(word_freq.most_common(n), columns=["word", "freq"])

    table.insert(0, 'Rank', range(1, len(table) + 1))
    return table

def plot_top_words(df, review):
    n = len(df)
    plt.figure(figsize=(10, 5))
    sns.barplot(x='freq', y='word', hue = "word",data=df, palette="Blues_r")

    plt.xlabel("Frequency", fontsize=12)
    plt.ylabel("Words", fontsize=12)
    plt.title(f"Top {n} Most Frequent Words: {review}", fontsize=12)
    plt.grid(axis='x', linestyle="--", alpha=0.5)
    plt.show()


In [ ]:
movies_tv_chunk =  movies_tv.iloc[:100_000] 
movies_tv_chunk

In [ ]:
movies_tv_chunk['process_review'] = movies_tv_chunk["reviewText"].apply(preprocess_text)

In [ ]:
bad_reviews, good_reviews = split_reviews(movies_tv, 1)

In [ ]:
TOP = 20

for header, dataset in zip(["All reviews", "Bad Reviews","Good Reviews"],[movies_tv,bad_reviews,good_reviews]):
    reviews = get_top_n_words(dataset,"process_review", TOP)
    plot_top_words(reviews, header)